
# <center>Analyzin world mortality with SQL  and Tableau </center>

## Index

1.	**Introduction**
2.	**Description of dataset**
3.	**Exploring mortality**
    1. Exploring mortality with SQL and Tableau
4.	**Analysis of mortality**
    1. Death by region
    2. Death by country
    3. Death by cause
    4. Death by age range
5. **Exploring Childhood Mortality**
    1. Child Mortality in the world
    2. General causes of death of children under five years of age
    3. Causes of death in the child population of mentioned countries
5. **Conclusions** 


## 1. Introduction

Mortality is a demographic indicator that indicates several hints about the world and individual countries’ situation. Analyzing causes of death might uncover trends and how they vary across different countries and world regions. With that information, public health departments and many organizations can trace strategies to prevent early death and improve their healthcare system which would then increase life quality. 
According to Hannah Ritchie, et al. (2018), around 56 million people die each year; but what caused their deaths? Is the cause of death different around the world? And why? Is death correlated to age? What can the number of deaths tell us? Is it true that older people die more often than young people? Are there places where this is not true? These kinds of questions are answered in this notebook using SQL.


## 2. Description of dataset


The data set used for this notebook was taken from [Our World in Data](https://ourworldindata.org/causes-of-death). The data consists of several tables whose columns are organized by countries, years and the total number of deaths for each registered cause. The years range from 1990 to 2019, the set of tables is organized by age rank, risk factor and the total number of deaths. 

## 3. Exploring Mortality

### Mortality Analysis with SQL and Tableau

In the path of a data analyst and a data scientist, SQL is one of the most powerful tools for cleaning and analyzing large amounts of data. SQL allows you to manage simple structured data to gain insights for your business. One of the advantages of this structured language that is its relative ease of use. 
Data analysis cannot be complete without a good visualization of the results, for this type of situation, Tableau offers a diverse set of tools for custom charts and dashboards, allowing the creation of amazing images.

The query below shows the PostgreSQL connection:

In [ ]:
%load_ext sql

In [ ]:
import os

In [ ]:
host = "localhost"
database = "Project_1"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

In [ ]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [ ]:
%sql $connection_string

## 4. Analisis of mortality

### Death by region

When talking about death, one of the question to comes in is what is the total number of deaths by region? and what of all regions register more deaths? 

The query below shows the running total number of death by region:

In [ ]:
%%sql 

SELECT "Entity", SUM(deaths) AS total_deaths
FROM 
(
SELECT "Entity", (
    SELECT SUM(t.f)
    FROM (VALUES
		  ("Deaths - Meningitis "),
		  ("Deaths - Alzheimers disease and other dementias"), 
		  ("Deaths - Parkinsons disease"),
		  ("Deaths - Nutritional deficiencies"),
		  ("Deaths - Malaria"),
		  ("Deaths - Drowning"),
		  ("Deaths - Interpersonal violence"),
		  ("Deaths - Maternal disorders"),
		  ("Deaths - HIV/AIDS"),
		  ("Deaths - Drug use disorders"),
		  ("Deaths - Tuberculosis"),
		  ("Deaths - Cardiovascular diseases"),
		  ("Deaths - Lower respiratory infections"),
		  ("Deaths - Neonatal disorders"),
		  ("Deaths - Alcohol use disorders"),
		  ("Deaths - Self-harm"),
		  ("Deaths - Exposure to forces of nature"),
		  ("Deaths - Diarrheal diseases"),
		  ("Deaths - Environmental heat and cold exposure"),
		  ("Deaths - Neoplasms"),
		  ("Deaths - Conflict and terrorism"),
		  ("Deaths - Diabetes mellitus"),
		  ("Deaths - Chronic kidney disease"),
		  ("Deaths - Poisonings"),
		  ("Deaths - Protein-energy malnutrition"),
		  ("Deaths - Terrorism (deaths)"),
		  ("Deaths - Road injuries"),
		  ("Deaths - Chronic respiratory diseases"),
		  ("Deaths - Cirrhosis and other chronic liver diseases"),
		  ("Deaths - Digestive diseases"),
		  ("Deaths - Fire heat and hot substances"),
		  ("Deaths - Acute hepatitis")	    
		 ) 
	AS t(f)
) AS deaths
FROM mortality."annual-number-of-deaths-by-cause"
WHERE "Code" is null and "Entity" like '%WH%' or "Entity" like '%WB%'
ORDER BY 1 DESC
) AS deaths
GROUP BY 1
ORDER BY 2 DESC

People could die because of several causes, this dataset count 32 causes of death. Taking that into account is worth knowing what is the main cause of death in the world.

The following query answers that question:

In [ ]:
%%sql 

WITH deaths_by_all_causes AS(
SELECT "Entity","Code", "Year",
UNNEST(ARRAY ['Meningitis',
			  'Alzheimers disease and other dementias',
			  'Parkinsons disease',
			  'Nutritional deficiencies',
			  'Malaria',
			  'Drowning',
			  'Interpersonal violence',
			  'Maternal disorders',
			  'HIV/AIDS',
			  'Drug use disorders',
			  'Tuberculosis',
			  'Cardiovascular diseases',
			  'Lower respiratory infections',
			  'Neonatal disorders',
			  'Alcohol use disorders',
			  'Self-harm',
			  'Exposure to forces of nature',
			  'Diarrheal diseases',
			  'Environmental heat and cold exposure',
			  'Neoplasms',
			  'Conflict and terrorism',
			  'Diabetes mellitus',
			  'Chronic kidney disease',
			  'Poisonings',
			  'Protein-energy malnutrition',
			  'Terrorism (deaths)',
			  'Road injuries',
			  'Chronic respiratory diseases',
			  'Cirrhosis and other chronic liver diseases',
			  'Digestive diseases',
			  'Fire heat and hot substances',
			  'Acute hepatitis'
			 ]) AS cause_of_deaths,
UNNEST(ARRAY ["Deaths - Meningitis ",
			  "Deaths - Alzheimers disease and other dementias",
			  "Deaths - Parkinsons disease",
			  "Deaths - Nutritional deficiencies",
			  "Deaths - Malaria",
			  "Deaths - Drowning",
			  "Deaths - Interpersonal violence",
			  "Deaths - Maternal disorders",
			  "Deaths - HIV/AIDS",
			  "Deaths - Drug use disorders",
			  "Deaths - Tuberculosis",
			  "Deaths - Cardiovascular diseases",
			  "Deaths - Lower respiratory infections",
			  "Deaths - Neonatal disorders",
			  "Deaths - Alcohol use disorders",
			  "Deaths - Self-harm",
			  "Deaths - Exposure to forces of nature",
			  "Deaths - Diarrheal diseases",
			  "Deaths - Environmental heat and cold exposure",
			  "Deaths - Neoplasms",
			  "Deaths - Conflict and terrorism",
			  "Deaths - Diabetes mellitus",
			  "Deaths - Chronic kidney disease",
			  "Deaths - Poisonings",
			  "Deaths - Protein-energy malnutrition",
			  "Deaths - Terrorism (deaths)",
			  "Deaths - Road injuries",
			  "Deaths - Chronic respiratory diseases",
			  "Deaths - Cirrhosis and other chronic liver diseases",
			  "Deaths - Digestive diseases",
			  "Deaths - Fire heat and hot substances",
			  "Deaths - Acute hepatitis"
			 ]) AS number_of_deaths
FROM mortality."annual-number-of-deaths-by-cause"
)
SELECT cause_of_deaths, SUM(number_of_deaths) AS total_deaths
FROM deaths_by_all_causes
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5

The query shows that the main cause of death in the world is **cardiovascular disease**, which is a general term for conditions affecting the heart or blood vessels. It's usually associated with a build-up of fatty deposits inside the arteries (atherosclerosis) and an increased risk of blood clots. Also, it can be associated with damage to arteries in organs such as the brain, heart, kidneys, and eyes.

Now the question is: Is it possible that this cause remains the main cause of death in all regions? 

The query below shows the top 5 causes of death for all regions organized by rank from 1 to 5. 

In [ ]:
%%sql


WITH deaths_by_all_causes AS(
SELECT "Entity","Code", "Year",
UNNEST(ARRAY ['Meningitis',
			  'Alzheimers disease and other dementias',
			  'Parkinsons disease',
			  'Nutritional deficiencies',
			  'Malaria',
			  'Drowning',
			  'Interpersonal violence',
			  'Maternal disorders',
			  'HIV/AIDS',
			  'Drug use disorders',
			  'Tuberculosis',
			  'Cardiovascular diseases',
			  'Lower respiratory infections',
			  'Neonatal disorders',
			  'Alcohol use disorders',
			  'Self-harm',
			  'Exposure to forces of nature',
			  'Diarrheal diseases',
			  'Environmental heat and cold exposure',
			  'Neoplasms',
			  'Conflict and terrorism',
			  'Diabetes mellitus',
			  'Chronic kidney disease',
			  'Poisonings',
			  'Protein-energy malnutrition',
			  'Terrorism (deaths)',
			  'Road injuries',
			  'Chronic respiratory diseases',
			  'Cirrhosis and other chronic liver diseases',
			  'Digestive diseases',
			  'Fire heat and hot substances',
			  'Acute hepatitis'
			 ]) AS cause_of_deaths,
UNNEST(ARRAY ["Deaths - Meningitis ",
			  "Deaths - Alzheimers disease and other dementias",
			  "Deaths - Parkinsons disease",
			  "Deaths - Nutritional deficiencies",
			  "Deaths - Malaria",
			  "Deaths - Drowning",
			  "Deaths - Interpersonal violence",
			  "Deaths - Maternal disorders",
			  "Deaths - HIV/AIDS",
			  "Deaths - Drug use disorders",
			  "Deaths - Tuberculosis",
			  "Deaths - Cardiovascular diseases",
			  "Deaths - Lower respiratory infections",
			  "Deaths - Neonatal disorders",
			  "Deaths - Alcohol use disorders",
			  "Deaths - Self-harm",
			  "Deaths - Exposure to forces of nature",
			  "Deaths - Diarrheal diseases",
			  "Deaths - Environmental heat and cold exposure",
			  "Deaths - Neoplasms",
			  "Deaths - Conflict and terrorism",
			  "Deaths - Diabetes mellitus",
			  "Deaths - Chronic kidney disease",
			  "Deaths - Poisonings",
			  "Deaths - Protein-energy malnutrition",
			  "Deaths - Terrorism (deaths)",
			  "Deaths - Road injuries",
			  "Deaths - Chronic respiratory diseases",
			  "Deaths - Cirrhosis and other chronic liver diseases",
			  "Deaths - Digestive diseases",
			  "Deaths - Fire heat and hot substances",
			  "Deaths - Acute hepatitis"
			 ]) AS number_of_deaths
FROM mortality."annual-number-of-deaths-by-cause"
WHERE "Code" IS null AND "Entity" LIKE '%WH%' OR "Entity" LIKE '%WB%'
)
SELECT ranked_causes_of_deaths.* FROM 
(
SELECT "Entity", cause_of_deaths, MAX(number_of_deaths) AS top_deaths,
RANK() OVER(PARTITION BY "Entity" ORDER BY MAX(number_of_deaths) DESC) AS rn
FROM deaths_by_all_causes
WHERE number_of_deaths IS NOT null 
GROUP BY 1,2 
ORDER BY 1, 3 DESC
	) ranked_causes_of_deaths
WHERE rn <= 5;

It is a well-known saying that an image says more than a thousand words, images make it easy to view trends, and for this reason, the output was organized in the graph below.

The graph shows the top 5 causes of death in each region. The graph shows that cardiovascular disease as the main cause of death prevails as a trend in almost all regions with more than 40% of total deaths except for the Africa Region whose main cause of death is HIV/AIDS.
The Africa Region includes Sub-Saharan Africa and North Africa. However, if the Arabic countries are separated from the non-Arabic ones and consider Sub-Saharan Africa and North Africa-Middle East, it is clear that Sub-Saharan Africa is responsible for the change in the trend and that the North Africa-Middle East region maintains cardiovascular disease as the principal cause of death. 

In [1]:
%%html
<div class='tableauPlaceholder' id='viz1673365659052' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph1MortalityProyect&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Graph1MortalityProyect&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph1MortalityProyect&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='es-ES' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1673365659052');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='889px';vizElement.style.maxWidth='1089px';vizElement.style.width='100%';vizElement.style.minHeight='351px';vizElement.style.maxHeight='551px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='889px';vizElement.style.maxWidth='1089px';vizElement.style.width='100%';vizElement.style.minHeight='351px';vizElement.style.maxHeight='551px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Death by country

In the same way, as with the regions, it is worth knowing which are the countries that register the most total deaths, for that only currently existing countries were taken into consideration, which means, countries like USSR were not taken into account.

The following query shows which of all countries registered more total deaths, the countries are organized  in a top 10:

In [ ]:
%%sql


SELECT "Entity","Code", SUM(deaths) AS total_deaths
FROM 
(
SELECT "Entity","Code", (
    SELECT SUM(t.f)
    FROM (VALUES
		  ("Deaths - Meningitis "),
		  ("Deaths - Alzheimers disease and other dementias"), 
		  ("Deaths - Parkinsons disease"),
		  ("Deaths - Nutritional deficiencies"),
		  ("Deaths - Malaria"),
		  ("Deaths - Drowning"),
		  ("Deaths - Interpersonal violence"),
		  ("Deaths - Maternal disorders"),
		  ("Deaths - HIV/AIDS"),
		  ("Deaths - Drug use disorders"),
		  ("Deaths - Tuberculosis"),
		  ("Deaths - Cardiovascular diseases"),
		  ("Deaths - Lower respiratory infections"),
		  ("Deaths - Neonatal disorders"),
		  ("Deaths - Alcohol use disorders"),
		  ("Deaths - Self-harm"),
		  ("Deaths - Exposure to forces of nature"),
		  ("Deaths - Diarrheal diseases"),
		  ("Deaths - Environmental heat and cold exposure"),
		  ("Deaths - Neoplasms"),
		  ("Deaths - Conflict and terrorism"),
		  ("Deaths - Diabetes mellitus"),
		  ("Deaths - Chronic kidney disease"),
		  ("Deaths - Poisonings"),
		  ("Deaths - Protein-energy malnutrition"),
		  ("Deaths - Terrorism (deaths)"),
		  ("Deaths - Road injuries"),
		  ("Deaths - Chronic respiratory diseases"),
		  ("Deaths - Cirrhosis and other chronic liver diseases"),
		  ("Deaths - Digestive diseases"),
		  ("Deaths - Fire heat and hot substances"),
		  ("Deaths - Acute hepatitis")	    
		 ) 
	AS t(f)
) AS deaths
FROM mortality."annual-number-of-deaths-by-cause"
WHERE "Code" IS NOT null AND "Code" NOT LIKE '%\_%'
ORDER BY 1 DESC
) AS deaths
GROUP BY "Entity", "Code"
ORDER BY 3 DESC
LIMIT 10;

We can see that the top ten countries are all countries with a high population, which is why it is not surprising that these countries are the ones that report the highest number of total deaths, but what are the leading causes of death in those countries? 

The following query gives the leading causes of death in those countries ranking from 1 to 5, where 1 is the cause with the most registered deaths and 5 is the last one. 

In [ ]:
%%sql

WITH deaths_by_all_causes AS(
SELECT "Entity","Code", "Year",
UNNEST(ARRAY ['Meningitis',
			  'Alzheimers disease and other dementias',
			  'Parkinsons disease',
			  'Nutritional deficiencies',
			  'Malaria',
			  'Drowning',
			  'Interpersonal violence',
			  'Maternal disorders',
			  'HIV/AIDS',
			  'Drug use disorders',
			  'Tuberculosis',
			  'Cardiovascular diseases',
			  'Lower respiratory infections',
			  'Neonatal disorders',
			  'Alcohol use disorders',
			  'Self-harm',
			  'Exposure to forces of nature',
			  'Diarrheal diseases',
			  'Environmental heat and cold exposure',
			  'Neoplasms',
			  'Conflict and terrorism',
			  'Diabetes mellitus',
			  'Chronic kidney disease',
			  'Poisonings',
			  'Protein-energy malnutrition',
			  'Terrorism (deaths)',
			  'Road injuries',
			  'Chronic respiratory diseases',
			  'Cirrhosis and other chronic liver diseases',
			  'Digestive diseases',
			  'Fire heat and hot substances',
			  'Acute hepatitis'
			 ]) AS cause_of_deaths,
UNNEST(ARRAY ["Deaths - Meningitis ",
			  "Deaths - Alzheimers disease and other dementias",
			  "Deaths - Parkinsons disease",
			  "Deaths - Nutritional deficiencies",
			  "Deaths - Malaria",
			  "Deaths - Drowning",
			  "Deaths - Interpersonal violence",
			  "Deaths - Maternal disorders",
			  "Deaths - HIV/AIDS",
			  "Deaths - Drug use disorders",
			  "Deaths - Tuberculosis",
			  "Deaths - Cardiovascular diseases",
			  "Deaths - Lower respiratory infections",
			  "Deaths - Neonatal disorders",
			  "Deaths - Alcohol use disorders",
			  "Deaths - Self-harm",
			  "Deaths - Exposure to forces of nature",
			  "Deaths - Diarrheal diseases",
			  "Deaths - Environmental heat and cold exposure",
			  "Deaths - Neoplasms",
			  "Deaths - Conflict and terrorism",
			  "Deaths - Diabetes mellitus",
			  "Deaths - Chronic kidney disease",
			  "Deaths - Poisonings",
			  "Deaths - Protein-energy malnutrition",
			  "Deaths - Terrorism (deaths)",
			  "Deaths - Road injuries",
			  "Deaths - Chronic respiratory diseases",
			  "Deaths - Cirrhosis and other chronic liver diseases",
			  "Deaths - Digestive diseases",
			  "Deaths - Fire heat and hot substances",
			  "Deaths - Acute hepatitis"
			 ]) AS number_of_deaths
FROM mortality."annual-number-of-deaths-by-cause"
WHERE "Code" IS NOT null AND "Code" NOT LIKE '%\_%'
),
top_countries AS (
SELECT "Entity","Code", SUM(deaths) AS total_deaths
FROM 
(
SELECT "Entity","Code", (
    SELECT SUM(t.f)
    FROM (VALUES
		  ("Deaths - Meningitis "),
		  ("Deaths - Alzheimers disease and other dementias"), 
		  ("Deaths - Parkinsons disease"),
		  ("Deaths - Nutritional deficiencies"),
		  ("Deaths - Malaria"),
		  ("Deaths - Drowning"),
		  ("Deaths - Interpersonal violence"),
		  ("Deaths - Maternal disorders"),
		  ("Deaths - HIV/AIDS"),
		  ("Deaths - Drug use disorders"),
		  ("Deaths - Tuberculosis"),
		  ("Deaths - Cardiovascular diseases"),
		  ("Deaths - Lower respiratory infections"),
		  ("Deaths - Neonatal disorders"),
		  ("Deaths - Alcohol use disorders"),
		  ("Deaths - Self-harm"),
		  ("Deaths - Exposure to forces of nature"),
		  ("Deaths - Diarrheal diseases"),
		  ("Deaths - Environmental heat and cold exposure"),
		  ("Deaths - Neoplasms"),
		  ("Deaths - Conflict and terrorism"),
		  ("Deaths - Diabetes mellitus"),
		  ("Deaths - Chronic kidney disease"),
		  ("Deaths - Poisonings"),
		  ("Deaths - Protein-energy malnutrition"),
		  ("Deaths - Terrorism (deaths)"),
		  ("Deaths - Road injuries"),
		  ("Deaths - Chronic respiratory diseases"),
		  ("Deaths - Cirrhosis and other chronic liver diseases"),
		  ("Deaths - Digestive diseases"),
		  ("Deaths - Fire heat and hot substances"),
		  ("Deaths - Acute hepatitis")	    
		 ) 
	AS t(f)
) AS deaths
FROM mortality."annual-number-of-deaths-by-cause"
WHERE "Code" IS NOT null AND "Code" NOT LIKE '%\_%'
ORDER BY 1 DESC
) AS deaths
GROUP BY "Entity", "Code"
ORDER BY 3 DESC
LIMIT 10
)
SELECT ranked_causes_of_deaths.* FROM (
SELECT a."Entity", a."Code", a.cause_of_deaths, MAX(number_of_deaths) AS top_deaths,
RANK() OVER(PARTITION BY a."Entity" ORDER BY MAX(number_of_deaths) DESC) AS rn
FROM deaths_by_all_causes a
JOIN top_countries b
ON a."Entity" = b."Entity"
WHERE number_of_deaths IS NOT null 
GROUP BY 1,2,3 
ORDER BY 1, 4 DESC
	) ranked_causes_of_deaths
WHERE rn <= 5;

To appreciate better those results, the graph below shows the top 5 deaths in the mentioned countries. Cardiovascular disease remains the main cause in almost all countries with at least 39% of total deaths in the set of 5 leading causes, except for Nigeria and Japan, in which the leading causes are diarrheal disease and neoplasm respectively. Also, there are clear differences between those countries with high economic development and those with lower economic development such as India, Nigeria, Pakistan, and Brazil, which register neonatal disorders as one of the main causes of death. 

In [2]:
%%html
<div class='tableauPlaceholder' id='viz1673365787527' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph2_1MortalityProject&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Graph2_1MortalityProject&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph2_1MortalityProject&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='es-ES' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1673365787527');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='920px';vizElement.style.maxWidth='1020px';vizElement.style.width='100%';vizElement.style.height='587px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='920px';vizElement.style.maxWidth='1020px';vizElement.style.width='100%';vizElement.style.height='587px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Death by cause

The following graph shows a global overview of the leading causes of death during the recorded years (1990-2019), the data were extracted with the following query, for ease, an additional table was created selecting only the 13 most frequent causes of death for all countries, and only 5 leading causes of death per year were taken into consideration. The output only shows the first 50 rows.

In [ ]:
%%sql

SELECT top5_deaths_by_year.* FROM (SELECT "Entity", "Code", "Year", cause_of_deaths, MAX(total_deaths) AS total_deaths,
RANK() OVER(PARTITION BY "Entity", "Year" ORDER BY MAX(total_deaths) DESC) AS rn
FROM mortality.top13_deaths_in_countries
GROUP BY 1,2,3,4
ORDER BY 1,5,6 DESC) top5_deaths_by_year
WHERE rn <= 5
ORDER BY 1,3,5 DESC
LIMIT 50

The graph below was made with the full output of the query above, it is possible to observe the trends of the main causes of death through the years, it is also possible to appreciate the occurrence of natural disasters such as Haiti where the 2004 Hurricane Jane and 2010 earthquake increased the number of deaths.

In [3]:
%%html
<div class='tableauPlaceholder' id='viz1673365851677' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph2MortalityProject&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Graph2MortalityProject&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph2MortalityProject&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='es-ES' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1673365851677');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='1020px';vizElement.style.maxWidth='1220px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='687px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='1020px';vizElement.style.maxWidth='1220px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='687px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Deaths by age rank

Death is the well-known termination of the cycle of life, a cycle that in humans can last up to a hundred years, which is why premature deaths hurt human pride. The age of death is an important factor to consider in mortality analysis.

The following query separates the number of deaths by age segment:

In [ ]:
%%sql

WITH deaths_by_age_group AS (
SELECT  
UNNEST (ARRAY ['Deaths in Age: 70+ years',
	   'Deaths in Age: 50-69 years',
	   'Deaths in Age: 15-49 years ',
	   'Deaths in Age: 5-14 years',
	   'Deaths in Age: Under 5']) AS deaths_by_age_group,
UNNEST (ARRAY ["Deaths - All causes - Age: 70+ years",
			  "Deaths - All causes - Age: 50-69 years",
			  "Deaths - All causes - Age: 15-49 years ",
			  "Deaths - All causes - Age: 5-14 years",
			  "Deaths - All causes - Age: Under 5"]) AS total
FROM mortality."number-of-deaths-by-age-group"
)
SELECT deaths_by_age_group, SUM(total) AS total
FROM deaths_by_age_group
GROUP BY 1
ORDER BY 2 DESC

It can be verified that in general terms the greatest number of deaths prevails in people over fifty years of age, followed by death in children under five years of age. However, this situation varies when compared by country. 

The query below gives an output for the total number of deaths for each country by age rank:

In [ ]:
%%sql

WITH deaths_by_age_group AS (
SELECT "Entity","Code", 
UNNEST (ARRAY ['Deaths Age: 70+ years',
	   'Deaths Age: 50-69 years',
	   'Deaths Age: 15-49 years ',
	   'Deaths Age: 5-14 years',
	   'Deaths Age: Under 5']) as deaths_by_age_group,
UNNEST (ARRAY ["Deaths - All causes - Age: 70+ years",
			  "Deaths - All causes - Age: 50-69 years",
			  "Deaths - All causes - Age: 15-49 years ",
			  "Deaths - All causes - Age: 5-14 years",
			  "Deaths - All causes - Age: Under 5"]) AS total
FROM mortality."number-of-deaths-by-age-group"
)
SELECT "Entity","Code",deaths_by_age_group, SUM(total) AS total
FROM deaths_by_age_group
WHERE "Code" IS NOT null AND "Code" NOT LIKE '%\_%'
GROUP BY 1,2,3
ORDER BY 1,4 DESC

The graph below shows the global trend that fewer and fewer people die at a young age. Globally in 1990 25% of the deaths were from children under 5 years old and for the year 2019 this percentage reduced to 9%, and the number of deaths in people older than 70 years changed from 33% in 1990 to 50% in 2019. The deaths of children between 5-14 years have changed little over those years, from 3% in 1990 to 1% in 2019. 

In [5]:
%%html
<div class='tableauPlaceholder' id='viz1673365927029' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph3MortalityProject&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Graph3MortalityProject&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph3MortalityProject&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='es-ES' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1673365927029');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='920px';vizElement.style.height='1087px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='920px';vizElement.style.height='1087px';} else { vizElement.style.width='100%';vizElement.style.height='1177px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## 5. Exploring Child Mortality

The study of child mortality occupies a special place in the field of demographic research, since it represents the negative component of population growth. Also, the moral aspect that the death of a child implies encourages the search for strategies to reduce this condition.

The following query gives the total number of deaths in children under 5 years old for each country:

In [ ]:
%%sql

WITH deaths_by_age_group AS (
SELECT "Entity","Code", 
UNNEST (ARRAY ['Deaths Age: 70+ years',
	   'Deaths Age: 50-69 years',
	   'Deaths Age: 15-49 years ',
	   'Deaths Age: 5-14 years',
	   'Deaths Age: Under 5']) as deaths_by_age_group,
UNNEST (ARRAY ["Deaths - All causes - Age: 70+ years",
			  "Deaths - All causes - Age: 50-69 years",
			  "Deaths - All causes - Age: 15-49 years ",
			  "Deaths - All causes - Age: 5-14 years",
			  "Deaths - All causes - Age: Under 5"]) as total
FROM mortality."number-of-deaths-by-age-group"
)
SELECT "Entity","Code",deaths_by_age_group, SUM(total) AS total
FROM deaths_by_age_group
WHERE "Code" IS NOT null AND "Code" NOT LIKE '%\_%'
GROUP BY 1,2,3 HAVING deaths_by_age_group = 'Deaths Age: Under 5'
ORDER BY 1,4 DESC

Organizing the output into a graph is possible to see that globally India registers the highest number of deaths in children under 5 years old. It is also possible to separate this number by region. In North America, the country with the most deaths registered is the United States and the one with the least is Greenland. On the other hand, in Latin America, the higher number of deaths occurs in Brazil, followed by Mexico and Haiti. In the African region, the country with the most deaths in Nigeria, followed by Ethiopia and the Democratic Republic of Congo. If the focus is on Europe, it is possible to see that Russia leads the list followed by Ukraine and the United Kingdom. In contrast in the Asiatic region, India leads followed by China and Pakistan. Finally, in Oceania, Papua Nueva Guinea has the higher number of deaths. 

# Grafico 4

In [6]:
%%html
<div class='tableauPlaceholder' id='viz1673365988007' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph4MortalityProject&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Graph4MortalityProject&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph4MortalityProject&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='es-ES' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1673365988007');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='400px';vizElement.style.maxWidth='900px';vizElement.style.width='100%';vizElement.style.minHeight='787px';vizElement.style.maxHeight='987px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='400px';vizElement.style.maxWidth='900px';vizElement.style.width='100%';vizElement.style.minHeight='787px';vizElement.style.maxHeight='987px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='2377px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

It is a fact that the trend of the mayority of death occurring after the age of 50 is not uniform throughout the world.
The following query gives a list of countries in which, the death under 5 years old have the highest number of registered

### Countries with majority of deaths occurring in children under five years of age

In [ ]:
%%sql


WITH deaths_by_age_group AS (
SELECT "Entity","Code", 
UNNEST (ARRAY ['Deaths: 70+ years',
	   'Deaths: 50-69 years',
	   'Deaths: 15-49 years ',
	   'Deaths: 5-14 years',
	   'Deaths: Under 5']) AS deaths_by_age_group,
UNNEST (ARRAY ["Deaths - All causes - Age: 70+ years",
			  "Deaths - All causes - Age: 50-69 years",
			  "Deaths - All causes - Age: 15-49 years ",
			  "Deaths - All causes - Age: 5-14 years",
			  "Deaths - All causes - Age: Under 5"]) AS total
FROM mortality."number-of-deaths-by-age-group"
)

SELECT countries_with_under5_mortality.* FROM (
SELECT "Entity","Code",deaths_by_age_group, SUM(total) AS total,
RANK() OVER(PARTITION BY "Entity" ORDER BY MAX(total) DESC) AS rn
FROM deaths_by_age_group
WHERE "Code" IS NOT null AND "Code" NOT LIKE '%\_%'
GROUP BY 1,2,3
ORDER BY 1,4 DESC
) AS countries_with_under5_mortality
WHERE deaths_by_age_group = 'Deaths: Under 5' AND rn = 1
ORDER BY 1

### General causes of death of children under five years of age

Exploring the causes of death in children under 5 years of age, it is possible to see that globally the main causes of death consist of neonatal (newborn infants less than 28 days old) disorders, infections, and congenital (from birth) defects. Lower respiratory infections have 25% of registered cases, followed by neonatal preterm birth at 23% and congenital birth defects at 20%. Children under 5 years of age are also susceptible to diarrheal infections, malnutrition, and nutritional deficiencies. It is possible to see that the main cause of death in this population vary across countries, being the nature of the deaths different for low and high-income countries, for example in the United States the main cause of death is neonatal preterm birth while in Haiti the main cause is a diarrheal disease. 

The graph below shows the total number of deaths by cause and the leading causes of death through years, only the first five causes of death per year were taken into account for all countries overall:

In [8]:
%%html
<div class='tableauPlaceholder' id='viz1673366956891' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph5MortalityProject&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Graph5MortalityProject&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gr&#47;Graph5MortalityProject&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='es-ES' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1673366956891');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='520px';vizElement.style.maxWidth='950px';vizElement.style.width='100%';vizElement.style.height='1087px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='520px';vizElement.style.maxWidth='950px';vizElement.style.width='100%';vizElement.style.height='1087px';} else { vizElement.style.width='100%';vizElement.style.height='1177px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## Conclusion

Data analysis allows for obtaining information that can offer practical and valuable solutions to existing problems, one of the most powerful tools for data analysis is SQL, which allows structured data to be thoroughly cleaned and analyzed. In this notebook, the task of analyzing mortality data using SQL and Tableau was proposed, it was achievable to analyze the data set to extract answers to the questions initially raised. 
In this way, one of the insights was that the main cause of death in the world is cardiovascular diseases, the same way it was possible to get the main countries with the most registered deaths and also their main causes of death.
Several main causes of death were taken into consideration to distinguish trends over the years for all countries, also in this analysis was attainable to observe natural disasters such as the earthquake in Haiti. 
When comparing by age range, could be discovered that fewer deaths of children under five years of age are registered annually, however, this trend was not absolute, and a list of countries where this trend is not met was set. Finally, the main causes of death in the child population and their variation over the years were determined.